# Inicialización

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Carga de datos

In [2]:
df = pd.read_csv('../datasets/users_behavior.csv')

# Exploración y corrección inicial de datos

In [3]:
# Primero le demos un vistazo al df
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


De lo que podemos observar las columnas son:
- `сalls` — número de llamadas
- `minutes` — duración total de la llamada en minutos
- `messages` — número de mensajes de texto
- `mb_used` — Tráfico de Internet utilizado en MB
- `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0)
  
Podemos notar un leve problema con el tipo de datos en nuestra tabla ya que las columnas `calls y messages` son *float* cuando deberian ser *int*. Tambien notamos como `is_ultra` es *int* cuando debería ser *boolean*.
  

Para nuestra suerte no vemos ningún valor ausente lo que facilita nuestro trabajo.

In [4]:
# Por ultimo veamos si hay algun duplicado en la tabla
df.duplicated().sum()

0

Con solo un problema fácil de tratar vamos a encargarnos de él y seguir con el proyecto.

In [5]:
# Simplemente reemplazamos los valores con la funcion astype
df = df.astype({'calls': int, 'messages': int, 'is_ultra': bool})

# Segmentación de datos
  
Dado que nuestro objetivo es predecir cual de los planes es más apropiado según el comportamiento del cliente es evidente que nuestro *target* es la columna `is_ultra` mientras que el resto de las columnas son las *features* sobre las cuales el modelo basará su predicción.
  
Vamos a usar la función `GridSearchCV` para la optimización de hiperparámetros la que nos permitirá reducir el sobreajuste de  nuestros modelos. Para lograr de mejor manera eso, vamos a dividir los datos en 2 partes: entrenamiento (75%) y pruebas (25%). 
  
De no usar `GridSearchCV` si habría que separar el dataset en 3 partes para tener un conjunto de validación.

In [6]:
# Primero separamos el df en las features y los target
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [7]:
# Primero segmentamos los datos en los datos de entrenamiento y los datos de validacion/prueba
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=56982)

Con nuestros datos segmentados de la forma correcta podemos proceder a entrenar nuestro modelo, aunque ahora nos vamos a enfocar en ver cual es el mejor modelo y cuales son los mejores hiperparámetros para lograr los mejores resultados.

# Prueba de modelos e hiperparámetros
  
En éste caso nos encontramos con una problemática de clasificación por lo que vamos a probar 3 modelos:
  
- Modelo de árbol de decisiones (DecisionTreeClassifier)
- Modelo de bosque alteatorio (RandomForestClassifier)
- Modelo de regresión logística (LogisticRegression)
  
Para poder refinar los diferentes modelos, obtener el mejor de cada uno y decidir cual será el modelo final vamos a utilizar el dataset de validación. Sin ir más lejos, vamos allá!

## Modelo de árbol de decisiones
  
Vamos a empezar con el árbol de decisiones ya que si bien el bosque aleatorio fabrica multiples árboles, nosotros vamos a poder obtener un conocimiento previo sobre cuales tipos de árboles presentan mejores predicciones para nuestro caso.
  
En vez de fabricar un nido enorme de bucles para probar y encontrar la mejor combinación de hiperparámetros, vamos a usar la funcion GridSearchCV. Vamos a explicar el motivo de cada parámetro que vamos a elegir.
- `criterion`: Vamos a elegir unicamente *entropy* ya que *gini* sirve principalemente si tenemos valores ausentes, que no es nuestro caso.
- `max_depth`: Vamos a probar profundidades del 1 al 10 ya que extendernos más allá de eso sería poco útil ya que nos puede traer sobreajuste y la misma función descarta automáticamente los modelos sobreajustados.
- `min_samples_split`: En éste caso probamos un rango de 4 a 12. 
- `min_samples_leaf`: En este caso es mejor mantenernos en valores aún más bajos para evitar de vuelta un modelo demasiado generalizado.
- `max_features`: Vamos a probar las funciones que nos permite *DecisionTreeClassifier* junto con la default que es *None*.

In [8]:
# Primero definimos los parametros que vamos a probar
params = {
    'random_state':[17203],
    'criterion':['entropy'],
    'max_depth':tuple(range(3,13)),
    'min_samples_split':tuple(range(2,11)),
    'min_samples_leaf':tuple(range(1,6)),
    'max_features':('auto', 'sqrt', 'log2',None)
}

In [9]:
# De ahí guardamos el grid en su variable para posterior trabajo
tree_grid = GridSearchCV(DecisionTreeClassifier(), param_grid= params, cv= 10, verbose= 1)

In [10]:
# Entrenamos la grid para encontrar las mejores combinaciónes
tree_grid.fit(features_train, target_train)

Fitting 10 folds for each of 1800 candidates, totalling 18000 fits


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': (3, 4, 5, 6, 7, 8, 9, 10, 11, 12),
                         'max_features': ('auto', 'sqrt', 'log2', None),
                         'min_samples_leaf': (1, 2, 3, 4, 5),
                         'min_samples_split': (2, 3, 4, 5, 6, 7, 8, 9, 10),
                         'random_state': [17203]},
             verbose=1)

In [11]:
# Finalmente le pedimos que nos de la combinación secreta
tree_grid.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features='auto',
                       min_samples_leaf=5, random_state=17203)

In [12]:
# Creamos entonces el modelo
tree_model = tree_grid.best_estimator_

Ahora vamos a observar como es la exactitud de nuestro modelo frente al conjunto de entrenamiento y al de prueba.

In [13]:
for feature, target, text in zip([features_train, features_test],
                                 [target_train, target_test],
                                 ['entrenamiento','prueba']):
    prediction = tree_model.predict(feature)
    
    accuracy = accuracy_score(target, prediction)
    
    print(f'Exactitud con el conjunto de datos de {text}: {accuracy}')

Exactitud con el conjunto de datos de entrenamiento: 0.8141078838174274
Exactitud con el conjunto de datos de prueba: 0.777363184079602


Bueno, esos resultados no son los mejores pero al menos pasan el umbral. No esperaba el mejor rendimiento dado que estamos trabajando con éste modelo que es relativamente simple. De todas formas vemos que más allá de usar la *grid* nos encontramos con un sobreajuste destacable.
  
De todas formas, pude notar que la *grid* no modificó el valor de *min_samples_split* y tomó el valor más alto de *min_samples_leaf*. Por lo tanto voy a llamar de vuelta a la función pero ésta vez le voy a pasar un rango de valores más alto en esos dos hiperparámetros.

In [14]:
# Definimos los parametros que vamos a probar
params_2 = {
    'random_state':[17203],
    'criterion':['entropy'],
    'max_depth':tuple(range(3,13)),
    'min_samples_split':tuple(range(5,14)),
    'min_samples_leaf':tuple(range(5,10)),
    'max_features':('auto', 'sqrt', 'log2',None)
}

In [15]:
# De ahí guardamos el grid en su variable para posterior trabajo
tree_grid_2 = GridSearchCV(DecisionTreeClassifier(), param_grid= params_2, cv= 10, verbose= 1)

In [16]:
# Veamos si ahora encontramos otras condiciones
tree_grid_2.fit(features_train, target_train)

Fitting 10 folds for each of 1800 candidates, totalling 18000 fits


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': (3, 4, 5, 6, 7, 8, 9, 10, 11, 12),
                         'max_features': ('auto', 'sqrt', 'log2', None),
                         'min_samples_leaf': (5, 6, 7, 8, 9),
                         'min_samples_split': (5, 6, 7, 8, 9, 10, 11, 12, 13),
                         'random_state': [17203]},
             verbose=1)

In [17]:
# Cambio el mejor modelo?
tree_grid_2.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features='auto',
                       min_samples_leaf=5, min_samples_split=12,
                       random_state=17203)

In [18]:
# Al parecer min_samples_split es mejor en 12, guardemos para evaluar como cambio
tree_model_2 = tree_grid_2.best_estimator_

In [19]:
tree_model_2.fit(features_train, target_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features='auto',
                       min_samples_leaf=5, min_samples_split=12,
                       random_state=17203)

In [20]:
for feature_2, target_2, text_2 in zip([features_train, features_test],
                                 [target_train, target_test],
                                 ['entrenamiento','prueba']):
    prediction_2 = tree_model_2.predict(feature_2)
    
    accuracy_2 = accuracy_score(target_2, prediction_2)
    
    print(f'Exactitud con el conjunto de datos de {text_2}: {accuracy_2}')
    

Exactitud con el conjunto de datos de entrenamiento: 0.8141078838174274
Exactitud con el conjunto de datos de prueba: 0.777363184079602


No voy a mentir, al ver ese resultado la primera vez pensé que simplemente había un problema con los nombres de las variables que me llebavan a llamar los valores anteriores y no los nuevos. Ante ese pensamiento simplemente agregué un  "_2"  a todo para asegurarme y mira mi sorpresa cuando veo que efectivamente el modelo sigue igual (hasta incluso modifiqué los nombres de las variables internas del *for* de pura paranoia). Por más extraño que me parezca, nos encontramos con que en nuestro modelo (y nuestros datos) el hiperparámetro *min_samples_split* no es muy influyente.
  
Tras visualizar los árboles de mis modelos encontré el motivo por el cual los resultados terminaron siendo iguales. Simplemente sucede que la cantidad de muestras en cada split nunca llega a superar las 12 y ni cerca.
  
De todas formas, algo que llegué a entender es que por más que me esfuerce en pulir el árbol de decisiones, parece que éste simplemente no pareciera ser el mejor. Habría que probar que resultados obtenemos con los otros modelos para sacar una conclusión definitiva respecto al tema.

## Modelo de bosque aleatorio
  
Basado en el árbol de decisión pero escalado a proporciones mayores y al hacer eso se le puede introducir aleatoriedad lo que le permite obtener una mayor adaptabilidad ante datos extraños.
  
El procedimiento será el mismo que con el árbol, usaremos la función GridSearchCV para obtener la mejor combinación de hiperparámetros y posteriormente los pondremos a prueba.

In [21]:
# Primero determinaremos sobre cuales hiperparámetros vamos a trabajar
params = {
    'n_estimators': [50,75,100],
    'criterion': ['entropy'],
    'max_depth': list(range(2,7)),
    'min_samples_split': list(range(2,10)),
    'min_samples_leaf': list(range(1,6)),
    'max_features': ['sqrt', 'log2', None],
    'random_state': [17203]
}

In [22]:
# Cargamos el grid en una variable
forest_grid = GridSearchCV(RandomForestClassifier(), param_grid= params, cv= 5, verbose= 1)

In [23]:
# Buscamos el mejor conjunto de h-parametros para nuestros datos
forest_grid.fit(features_train, target_train)

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['entropy'], 'max_depth': [2, 3, 4, 5, 6],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [50, 75, 100],
                         'random_state': [17203]},
             verbose=1)

In [24]:
# Finalmente vemos cual es
forest_grid.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=6, max_features='sqrt',
                       min_samples_split=8, n_estimators=75,
                       random_state=17203)

Nos detengamos a analizar el modelo obtenido de la grid. Lo primero que quiero observar es si alguno de los h-parametros tomó el valor más alto del rango que le dí. Al ver con ese foco vemos que solo *max_depth* cumple con esa condición, mientras tanto vemos que *min_samples_leaf* ni siquiera fue especificado por lo que podemos deducir que no tiene un impacto grande en nuestro modelo con nuestros datos. Antes de probar otra combinación de h-parametros, ya que tarda tanto en ejecutarse, vamos a ver la exactitud del modelo en cuestión.

In [25]:
# Cargamos el modelo en una variable
forest_model = forest_grid.best_estimator_

In [26]:
# Lo entrenamos
forest_model.fit(features_train, target_train)

RandomForestClassifier(criterion='entropy', max_depth=6, max_features='sqrt',
                       min_samples_split=8, n_estimators=75,
                       random_state=17203)

In [27]:
# Y finalmente lo ponemos a prueba
for feature, target, text in zip([features_train, features_test],
                                 [target_train, target_test],
                                 ['entrenamiento','prueba']):
    
    prediction = forest_model.predict(feature)
    
    accuracy = accuracy_score(target, prediction)
    
    print(f'Exactitud con el conjunto de datos de {text}: {accuracy}')

Exactitud con el conjunto de datos de entrenamiento: 0.8327800829875519
Exactitud con el conjunto de datos de prueba: 0.7761194029850746


Ahora vamos a efectuar las correcciónes que destacamos previamente y vamos a hacer un *grid* nuevo.

In [28]:
# Primero designamos los parámetros
params = {
    'n_estimators': [50,75,100],
    'criterion': ['entropy'],
    'max_depth': list(range(6,13)),
    'min_samples_split': list(range(2,10)),
    'max_features': ['sqrt', 'log2', None,2],
    'random_state': [17203]
}

# De ahí creamos el grid con los nuebos parametros
forest_grid_2 = GridSearchCV(RandomForestClassifier(), param_grid= params, cv= 8, verbose= 1)

In [ ]:
# Ejecutamos el fit en el grid para nuestros datos
forest_grid_2.fit(features_train, target_train)

Fitting 8 folds for each of 672 candidates, totalling 5376 fits


In [ ]:
# Y veamos que nos dió
forest_grid_2.best_estimator_

In [ ]:
# Cargamos el nuevo modelo en otra variable
forest_model_2 = forest_grid_2.best_estimator_

In [ ]:
# Entrenamos el nuevo modelo
forest_model_2.fit(features_train, target_train)

In [ ]:
# Y vemos la exactitud con los conjuntos de prueba y entrenamiento
for feature, target, text in zip([features_train, features_test],
                                 [target_train, target_test],
                                 ['entrenamiento','prueba']):
    
    prediction = forest_model_2.predict(feature)
    
    accuracy = accuracy_score(target, prediction)
    
    print(f'Exactitud con el conjunto de datos de {text}: {accuracy}')

Okay, dejemos bien escrito los valores para poder compararlos como se debe:
- Modelo 1: 
    - Exactitud con el conjunto de datos de entrenamiento: 0.83278
    - Exactitud con el conjunto de datos de prueba: 0.77611
    - Diferencia entre los dos: 0.05667
  
- Modelo 2:
    - Exactitud con el conjunto de datos de entrenamiento: 0.85518
    - Exactitud con el conjunto de datos de prueba: 0.77860
    - Diferencia entre los dos: 0.07658 

Al ver los datos uno encima del otro nos facilita ver los cambios que tienen los resultados. Podemos ver como en el segundo modelo se alcanzó en ambos casos una mayor exactitud que en el primero, aunque cabe destacar que tambien se aumentó la distancia entre la exactitud de los datos de entrenamiento y los de prueba. Eso nos puede indicar que el segundo modelo posiblemente esté un poco mas sobreajustado que el primero, aunque los dos están sobreajustados hasta cierto punto.
  
A diferencia del árbol de decisiones, acá si son diferentes los resultados de los modelos por lo que tengo que elegir uno para comparar al final. Para éste caso me encuentro en la disyuntiva ya que el objetivo de éste estudio es encontrar el modelo con la mayor exactitud posible, que en nuestro caso sería el segundo. Pero el problema que se me viene a la mente es que según los resultados que tenemos arriba el segundo modelo tiene mayor exactitud en el conjunto de prueba pero a la vez posee una mayor diferencia con la exactitud del conjunto de entrenamiento lo que me indica que está más sobreajustado. De todas formas, las reglas son las reglas y para respetarlas debemos elegir entonces el segundo modelo.

## Modelo de regresión logística
  
Mantendremos el mismo modus operandi que tuvimos con los otros dos modelos y usaremos un grid para encontrar la mejor combinación de h-parámetros para nuestro modelo acorde a nuestros datos.

In [ ]:
# Primero vamos a determinar los h-parámetros que vamos a probar
params = {
    'C': np.logspace(-4,4,30),
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': list(range(1000,2100,50)),
    'random_state': [17203]
}

logistic_grid = GridSearchCV(LogisticRegression(), param_grid= params, cv= 10, verbose= 1)

In [ ]:
# Con el grid establecido vamos a probar cuales son los mejores h-parametros
logistic_grid.fit(features_train, target_train)

In [ ]:
# Vemos entonces el mejor resultado
logistic_grid.best_estimator_

In [ ]:
# Lo guardamos una variable
logistic_model = logistic_grid.best_estimator_

In [ ]:
# Y lo entrenamos
logistic_model.fit(features_train, target_train)

In [ ]:
# Finalmente vemos la exactitud con los conjuntos de prueba y entrenamiento
for feature, target, text in zip([features_train, features_test],
                                 [target_train, target_test],
                                 ['entrenamiento','prueba']):
    
    prediction = logistic_model.predict(feature)
    
    accuracy = accuracy_score(target, prediction)
    
    print(f'Exactitud con el conjunto de datos de {text}: {accuracy}')

Un poco me esperaba ésto. El modelo de regresión logística es caracterizado por ser eficiente y rápido pero no muy preciso. Una cosa que tambien se puede afirmar sobre éste modelo es que lidia muy bien con el sobreajuste y lo vemos claramente ya que presenta valores muy similares tanto en los datos de entrenamiento como en los de prueba. En resumen, es un buen modelo para algo que no requiera exactitud... triste que no es nuestro caso. 

## Elección final y prueba de cordura
  
Ahora llegó el momento de elegir al ganador de éste evento. Primero, recapitulemos los resultados de cada modelo:
1. **Modelo de árbol de decisiones**
    - Exactitud con el conjunto de datos de entrenamiento: 0.81410
    - Exactitud con el conjunto de datos de prueba: 0.77736
2. **Modelo de bosque aleatorio**
    - Exactitud con el conjunto de datos de entrenamiento: 0.85518
    - Exactitud con el conjunto de datos de prueba: 0.77860
3. **Modelo de regresión logística**
    - Exactitud con el conjunto de datos de entrenamiento: 0.69834
    - Exactitud con el conjunto de datos de prueba: 0.69029
  
Cabe destacar que estamos eligiendo los mejores valores de cada modelo. En resumen, en números netos el modelo con mayor exactitud es el de bosque aleatorio (y con el tiempo que tomó, más le vale). Podemos avanzar tranquilos ya que pasamos el umbral por 0.0286 que no es mucho, pero es trabajo honesto.
  
Ya lo destaqué antes, pero a pesar de que elijo el modelo de bosque aleaetorio lo hago conscientemente de que es un modelo con un grado no menor de sobreajuste por lo que ponerlo a prueba con datos nuevos puede devolver una exactitud menor a la que vemos en las pruebas. Con esa aclaración lista, sigamos.
  
Con el modelo determinado, vamos a guardar el modelo en una variable aparte para ordenar bien todo y realizar la prueba de cordura.

In [ ]:
final_model = forest_model_2

Ahora a explicar como haremos la prueba de cordura. Si bien podriamos crear un modelo que simplememte asigne 0 y 1 con la misma probabilidad, decir que tiene una exactitud menor que nuestro modelo y quedarnos con eso... Eso no sería algo que me deje conforme por lo que vamos a observar la distribucion de la columna `is_ultra` (nuestro target) en todo el df y vamos a comparar con la distribución de la predicción de nuestro modelo.

In [ ]:
# Primero veamos la distribución del df completo
pd.concat(
    [df['is_ultra'].value_counts(normalize= True),
    df['is_ultra'].value_counts()], axis=1
)

In [ ]:
# Ahora veamos como se comporta nuestro modelo
pd.concat(
[pd.Series(final_model.predict(df.drop(['is_ultra'],axis=1))).value_counts(normalize= True),
pd.Series(final_model.predict(df.drop(['is_ultra'],axis=1))).value_counts()], axis= 1
)

In [ ]:
accuracy_score(df['is_ultra'],final_model.predict(df.drop(['is_ultra'],axis=1)))

Al ver unicamente las distribuciones (y principalmente las normalizadas) sentí que el modelo tuvo resultados peores de los esperados, pero al ver los numeros netos y el *accuracy_score* me volvió la esperanza al cuerpo. Al parecer nuestro modelo tiene muchos falsos negativos y al ver el accuarcy score se ve como éste se encuentra un poco por debajo del obtenido al predecir los datos de entrenamiento. Por supuesto, ésto tiene sentido ya que la mayoria de los datos del df son los de entrenamiento.
  
Entonces, con una diferencia normalizada de 0.11 podemos tener un buen grado de certeza de que nuestro modelo si pasa una prueba de cordura y es mejor que simplemente asignar aleatoriamente los resultados.

# Conclusión
  
Trabajar con éste proyecto fue ciertamente una prueba de... paciencia. Lo primero que tenemos que destcar (y agradecer) es la limpieza de los datos que nos tocaron ya que no tuvimos que realizar ningun tipo de intervención y pudimos pasar directamente al trabajo de verdad.
  
Originalmente planeaba trabajar con 3 conjuntos de datos: uno de entrenamiento, uno de validación y uno de prueba. El segundo iba a estar dedicado a ayudarnos a refinar los hiperparámetros de los modelos de clasificación para así posteriormente evaluar los modelos en sí con el conjunto de pruebas. Como vimos, eso no fue lo que hice ya que no mucho después de mi investigación para la refinación de los hiperparámetros me encontré con la función *GridSearchCV*. Ésta función logra de manera más eficiente y con mayor precisión el trabajo que mis nidos de bucles iban a lograr, y encima de eso también hace verificaciones cruzadas entre los datos. Gracias a ella, pude directamente dedicar el 75% de los datos al entrenamiento y el otro 25% a prueba. No tengo dudas de que eso ayudó a lograr un modelo de mejor calidad y posiblemente en un menor tiempo.
  
A la hora de trabajar con los modelos optamos por probar 3 modelos diferentes:
1. **Árbol de decisiones**: Un modelo simple y rápido que va separando en base a las características para finalmente culminar en la clasificación a tomar (1 o 0).
2. **Bosque aleatorio**: Un modelo que parte con el árbol de decisiones pero que expande la idea inicial creando múltiples árboles diferentes "especializados" en diferentes grupos de categorías y posteriormente toma una decisión ponderando los diferentes resultados. Como se ha de esperar, es un modelo con una alta carga computacional y un rendimiento logarítmico.
3. **Regresión logística**: Nacido en 1958 a partir de la regresión lineal pero aplicado sobre un eje logarítmico en función de p/1-p lo que le permite obtener matemáticamente resultados categóricos a partir de valores continuos o discretos. Debido a su naturaleza matemática es muy eficiente computacionalmente pero con una capacidad de predicción que muchas veces queda corto.
  
Al trabajar con los modelos nos enteramos que nuestros datos presentaban unas características muy peculiares que se ven al entrar al detalle de los dos modelos de árbol que hicimos. Al ver de manera gráfica los árboles pudimos notar como éstos no logran refinar de manera uniforme los datos y éstos terminan con multiples hojas con una enorme cantidad de valores (una hoja tenia +1700 valores). Esa experiencia demostró que quizas el árbol de decisiones no era el mejor modelo para nuestros datos aunque si era suficiente ya que cumplía con el umbral establecido de 0,75.
  
Posteriormente trabajamos con el modelo de bosque aleatorio que demostró sin lugar a dudas que éste era el modelo más pesado computacionalmente pero que finalmente logró los mejores resultados alcanzando una exactitud con el conjunto de pruebas de un 0,7786. Cabe destacar que éste modelo presentaba el mayor sobreajuste de todos pero como nos preocupamos por la exactitud final nos quedamos con él.
  
Finalmente le dimos una prueba al modelo de regresión logística el cual se ejecutó bastante rápido y me obligó a leer muy detalladamente la documentación por unos errores que no llegaron al proyecto final. Como era de esperar el mismo presentó la menor exactitúd de los tres pero a su vez demostró el menor sobreajuste de todos. Éste no logró superar el umbral de 0,75.
  
En conclusión, logramos obtener un modelo que supere el umbral establecido por 0.0286 y que al ponerlo a someterlo a una prueba de cordura éste logró superarla aunque demostrando una tendencia a los falsos negativos. Y que se puede mejorar? Para empezar, nunca vienen mal más filas en nuestros datos, pero eso muchas veces no es posible, por lo que se podría buscar obtener nuevas columnas como si un cliente sobrepasó el límite mensual de datos/mensajes/llamadas o el monto total que cierto cliente pagó. 